# Scence

As all of us now, in many sitcomes and seinfled spesificly the show is a series of scene.  
Many times, two seperate scence are completly different (for example Gorege and Jerry are in the street, next scene they sit togather in a resturant).  
Since we excpt following sentances will help us better predict, we shouldn't consider different scence as following sentacnes.  
Moreover, we remmber kramer break in to the room and the audince laughing.  

After watching enough senfiled eppsiodes we now that sometimes we have this _[transiation music](https://www.youtube.com/watch?v=EP2GMyQdw78)_ it's not always the case, and we did see different timings of this interlude.

In [1]:
from seinfeld_playground import *
from train_utils import split_train_test
from compare_models import plot_confusion_matrix, compare_models_roc_curve
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

from bokeh.io import output_notebook, show
from bokeh.plotting import figure
output_notebook()
%load_ext autoreload
%autoreload 2

df = load_corpus()

Loading BokehJS ...

Let's start with finiding the time between following sentances, we shall remmber that new episodes should have time zero

In [2]:
df.loc[:, 'time_from_prev'] = np.array([0] + [df.start[i] - df.end[i - 1]
                                       if df.episode_num[i] == df.episode_num[i - 1] else 0
                                       for i in range(1, len(df.start))])


In [3]:
p = figure(plot_height=300, tools="pan,box_zoom,reset, save")

hist, edges = np.histogram(df.time_from_prev, bins=np.arange(-0.1,50,0.2))
p.quad(top=hist, bottom=0, left=edges[:-1], right=edges[1:], alpha=0.4)

show(p)

What do we see?  
As excpected, most of the following sentances occur after < 1sec.  
But we are intrested in the long tail, it's very weird to see distance of > 40sec, let's look at them

In [4]:
df.iloc[[j for i in df[(df.time_from_prev > 40)].index for j in [i-1, i]]].head(6)

,avg_word_length,character,end,episode_name,episode_num,is_funny,laugh_time,length,line_num,num_words,season,start,total_lines,txt,global_episode_num,time_from_prev
7147,4.000000,NaN,1255.228,The Smelly Car,21,True,1255.3,1.517,461,2.0,4,1253.711,475,Sauce me.,15,0.000
7148,2.800000,NaN,1302.299,The Smelly Car,21,False,NaN,2.125,462,5.0,4,1300.174,475,Why do we need BO?,15,44.946
11216,5.400000,NaN,426.385,The Barber,8,True,451.5,2.694,176,5.0,5,423.691,445,"Well, we'll straighten him out.",24,0.074
11217,3.285714,NaN,472.437,The Barber,8,False,NaN,1.533,177,7.0,5,470.904,445,What do you do there all day?,24,44.519
21258,4.000000,NaN,1283.827,The Jimmy,19,True,1284.7,5.299,441,11.0,6,1278.528,454,I wanna dedicate this song to a very courageo...,44,0.741
21259,4.600000,KRAMER,1390.368,The Jimmy,19,False,NaN,2.398,442,5.0,6,1387.970,454,"Hey, got the new Penthouse.",44,104.143


After watching some episodes that's look allright.  
It's really a new scene, and it there is that match seconds without speaking.  
  
  
We prefer to have false negative than false positive --> we prefer to have two scences togather than to break in a middle of a scene, so we will pick the threshold a bit high, let's say 5 seconds

In [25]:
df.episode_num.sample(1).values[0]

16

In [29]:
# df.loc[:, ['new_scene']] = df['time_from_prev'] > 5
# # Other than that, if it's a new episode we want new scene, used heuristic for that
# df.loc[(df.time_from_prev == 0) & (df.start <= 2), ['new_scene']] = True

df[(df.episode_num == df.episode_num.sample(1).values[0]) & (df.season == df.season.sample(1).values[0])]

,avg_word_length,character,end,episode_name,episode_num,is_funny,laugh_time,length,line_num,num_words,season,start,total_lines,txt,global_episode_num,time_from_prev,new_scene
37151,4.400000,NaN,7.534,The Yada Yada,19,False,NaN,1.260,0,5.0,8,6.274,496,You're on a desert island.,78,0.000,False
37152,3.636364,NaN,9.835,The Yada Yada,19,False,NaN,2.227,1,10.0,8,7.608,496,You can bring five books. Which five do you t...,78,0.074,False
37153,4.000000,NaN,11.535,The Yada Yada,19,True,12.0,1.624,2,5.0,8,9.911,496,I gotta read five books?,78,0.076,False
37154,4.333333,NaN,13.839,The Yada Yada,19,False,NaN,2.227,3,3.0,8,11.612,496,"All right, one.",78,0.077,False
37155,3.500000,NaN,16.405,The Yada Yada,19,False,NaN,1.590,4,2.0,8,14.815,496,Come on.,78,0.976,False
37156,4.600000,NaN,18.074,The Yada Yada,19,False,NaN,1.590,5,5.0,8,16.484,496,I got it. Three Musketeers.,78,0.079,False
37157,5.000000,NaN,19.567,The Yada Yada,19,False,NaN,1.415,6,3.0,8,18.152,496,You've read that?,78,0.078,False
37158,3.857143,NaN,20.982,The Yada Yada,19,True,21.3,1.415,7,7.0,8,19.567,496,"No, I'm saving it for the island.",78,0.000,False
37159,4.222222,NaN,23.783,The Yada Yada,19,False,NaN,2.728,8,8.0,8,21.055,496,"All right, let's start this whole thing over.",78,0.073,False
37160,5.571429,NaN,26.688,The Yada Yada,19,False,NaN,2.830,9,6.0,8,23.858,496,"Best Chamberlain: Wilt, Richard, or Neville?",78,0.075,False


Not fantastic, we still see FP